In [ ]:
import numpy as np
from cellpose import models
import napari
import os
from scripts.sample_db import SampleDB
from tifffile import imwrite, imread

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

# Loading experiment
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp)

# Making shortcuts of sample parameters/information
sample = exp.sample
raw_path = exp.paths["raw_path"]
anatomy_path = exp.paths["anatomy_path"]
em_path = exp.paths["em_path"]
n_planes = exp.params_lm.n_planes
n_frames = exp.params_lm.n_frames
n_slices = exp.params_lm.lm_stack_range
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
trial_paths = os.listdir(os.path.join(raw_path, "trials"))

# Define the path for the preprocessed folder
preprocessed_folder = os.path.join(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3", "preprocessed")
os.makedirs(preprocessed_folder, exist_ok=True)

# Define the path for the masks folder
masks_folder = os.path.join(preprocessed_folder, "..", "masks")
os.makedirs(masks_folder, exist_ok=True)

In [2]:
import numpy as np
import os
from tifffile import imread
import pandas as pd
import napari

# Define paths
preprocessed_folder = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed'
masks_folder = os.path.join(preprocessed_folder, "..", "masks")

# Load transformation parameters
trial_paths = os.listdir(preprocessed_folder)
trial_paths = [tp for tp in trial_paths if tp.startswith("transformed_plane_")]

# Load masks
n_planes = 8  # Update this with the actual number of planes
all_masks = []

for plane in range(n_planes):
    plane_str = f"{plane:02d}"
    mask_path = os.path.join(masks_folder, f"mask_plane_{plane_str}.tif")
    masks = imread(mask_path)
    all_masks.append(masks)

# Convert lists to numpy arrays
all_masks = np.array(all_masks)

In [5]:
trial_paths

['transformed_plane_03_20220426_RM0008_130hpf_fP1_f3_t1_o3Ctrl_001_.tif',
 'transformed_plane_00_20220426_RM0008_130hpf_fP1_f3_t3_o5GCA_001_.tif',
 'transformed_plane_04_20220426_RM0008_130hpf_fP1_f3_t2_o7Cad_001_.tif',
 'transformed_plane_07_20220426_RM0008_130hpf_fP1_f3_t1_o6TDCA_001_.tif',
 'transformed_plane_07_20220426_RM0008_130hpf_fP1_f3_t3_o3Ctrl_001_.tif',
 'transformed_plane_05_20220426_RM0008_130hpf_fP1_f3_t3_o4TCA_001_.tif',
 'transformed_plane_05_20220426_RM0008_130hpf_fP1_f3_t1_o1Ala_001_.tif',
 'transformed_plane_05_20220426_RM0008_130hpf_fP1_f3_t1_o3Ctrl_001_.tif',
 'transformed_plane_06_20220426_RM0008_130hpf_fP1_f3_t2_o5GCA_001_.tif',
 'transformed_plane_02_20220426_RM0008_130hpf_fP1_f3_t3_o7Cad_001_.tif',
 'transformed_plane_03_20220426_RM0008_130hpf_fP1_f3_t1_o5GCA_001_.tif',
 'transformed_plane_07_20220426_RM0008_130hpf_fP1_f3_t1_o7Cad_001_.tif',
 'transformed_plane_07_20220426_RM0008_130hpf_fP1_f3_t2_o3Ctrl_002_.tif',
 'transformed_plane_06_20220426_RM0008_130hpf_

In [ ]:
import pandas as pd
import numpy as np

import os
from scripts.sample_db import SampleDB
from tifffile import imwrite, imread

# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

# Loading experiment
sample_id = '20220426_RM0008_130hpf_fP1_f3'
exp = sample_db.get_sample(sample_id)
print(exp)


# Making shortcuts of sample parameters/information
sample = exp.sample
raw_path = exp.paths["raw_path"]
anatomy_path = exp.paths["anatomy_path"]
em_path = exp.paths["em_path"]
n_planes = exp.params_lm.n_planes
n_frames = exp.params_lm.n_frames
n_slices = exp.params_lm.lm_stack_range
doubling = 2 if exp.params_lm.doubling else 1

# Getting paths of the trial acquisitions
# Getting paths of the trial acquisitions
trial_paths = [os.path.splitext(path)[0] for path in os.listdir(os.path.join(raw_path, "trials"))]
print(trial_paths)
# Define the path for the preprocessed folder
preprocessed_folder = os.path.join(r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3", "preprocessed")
os.makedirs(preprocessed_folder, exist_ok=True)

# Define the path for the masks folder
masks_folder = os.path.join(preprocessed_folder, "..", "masks")
os.makedirs(masks_folder, exist_ok=True)


# Load masks
n_planes = 8  # Update this with the actual number of planes
all_masks = []

for plane in range(n_planes):
    plane_str = f"{plane:02d}"
    mask_path = os.path.join(masks_folder, f"mask_plane_{plane_str}.tif")
    masks = imread(mask_path)
    all_masks.append(masks)

# Convert lists to numpy arrays
all_masks = np.array(all_masks)

# Create folder for saving fluorescence data
traces_folder = os.path.join(preprocessed_folder, "..", "traces")
os.makedirs(traces_folder, exist_ok=True)

In [17]:
# Calculate fluorescence intensity


for plane in range(n_planes):
    plane_str = f"{plane:02d}"


    print(all_masks.shape)
    
    for trial_idx, trial_path in enumerate(trial_paths):
        movie_path = os.path.join(preprocessed_folder, f"transformed_plane_{plane_str}_{trial_path}.tif")
        movie = imread(movie_path)
        mask = all_masks[plane, trial_idx, :, :]

        trial_fluorescence = {}
        for label in np.unique(mask):
            if label != 0:
                label_mask = mask == label

                fluorescence_values = movie[:, label_mask].mean(axis=1)
                trial_fluorescence[label] = fluorescence_values
        
        # Save the fluorescence data for the current plane and trial
        traces_file_path = os.path.join(traces_folder, f"traces_plane_{plane_str}_{trial_path}.npy")
        np.save(traces_file_path, trial_fluorescence)
        print(f"Fluorescence data saved at {traces_file_path}")

print("Fluorescence intensities calculated and saved.")

(8, 24, 256, 512)
Fluorescence data saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_00_20220426_RM0008_130hpf_fP1_f3_t1_o1Ala_001_.npy
Fluorescence data saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_00_20220426_RM0008_130hpf_fP1_f3_t1_o2Ser_001_.npy
Fluorescence data saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_00_20220426_RM0008_130hpf_fP1_f3_t1_o3Ctrl_001_.npy
Fluorescence data saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_00_20220426_RM0008_130hpf_fP1_f3_t1_o4TCA_001_.npy
Fluorescence data saved at \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_00_20220426_RM0008_130

In [71]:
import matplotlib.pyplot as plt
traces_file_path = r"\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-04-26\f3\preprocessed\..\masks\traces\traces_plane_03_20220426_RM0008_130hpf_fP1_f3_t1_o6TDCA_001_.npy"

trial_fluorescence = np.load(traces_file_path, allow_pickle=True)

np_traces = np.array(trial_fluorescence)
#np_traces = np.array(list(trial_fluorescence.values()))
for trace in np_traces:
    plt.plot(trace)
mean_traces = np.mean(np_traces, axis=0)    
plt.plot(mean_traces)


TypeError: iteration over a 0-d array

In [80]:
np.array(trial_fluorescence.item())

array({1: array([8125.83364627, 8126.58474688, 8126.61167168, 8137.20696396,
       8134.6220686 , 8126.09776821, 8125.6692792 , 8126.07980913,
       8141.63326198, 8140.56702253, 8132.70143353, 8147.02692937,
       8134.76623167, 8135.31032951, 8136.26110395, 8140.6581946 ,
       8133.74431441, 8144.66102041, 8132.36649147, 8138.58410593,
       8135.36834677, 8135.11598547, 8134.57029175, 8135.96114184,
       8134.21918517, 8135.38670419, 8132.76092249, 8136.15778607,
       8135.57607658, 8137.76003033, 8135.74795732, 8135.72307575,
       8132.1766147 , 8131.24252758, 8138.32659079, 8138.42077593,
       8132.49665535, 8128.71478038, 8129.89384267, 8131.28658158,
       8128.57598413, 8133.66044215, 8133.66838402, 8133.07036869,
       8137.32126457, 8130.07103277, 8135.13782289, 8130.45482828,
       8136.5187939 , 8140.8043703 , 8137.34278448, 8143.54360176,
       8131.02077376, 8142.6981465 , 8134.20415769, 8131.74727362,
       8131.85304463, 8137.46724377, 8133.644499  , 

In [1]:

# Convert fluorescence intensities to a DataFrame and save as CSV
for key, value in fluorescence_intensities.items():
    df = pd.DataFrame(value)
    df.to_csv(os.path.join(masks_folder, f"{key}_fluorescence.csv"))

print("Fluorescence intensities calculated and saved.")

FileNotFoundError: [Errno 2] No such file or directory: '\\\\tungsten-nas.fmi.ch\\tungsten\\scratch\\gfriedri\\montruth\\2P_RawData\\2022-04-26\\f3\\preprocessed\\transformed_plane_00_transformed_plane_03_20220426_RM0008_130hpf_fP1_f3_t1_o3Ctrl_001_.tif'